# 1. 데이터 정리
## 1-0. 사용 데이터
- 카카오 아레나 `train.json`
- 카카오 아레나 `song_meta.json`
- 카카오 아레나 `genre_gn_all.json`
- 회의에서 정한 `사용할 장르만 정리`한 데이터
- 회의에서 정한 `태그 - 감정` 매칭 데이터

## 1-1. 모듈 import, data load

In [2]:
import pandas as pd
import numpy as np
import json
from collections import Counter

In [ ]:
# train data
train = pd.read_json('data.json')
train = pd.DataFrame(train)
train.head()

In [ ]:
# song meta data
songs = pd.read_json('data.json')
songs = pd.DataFrame(songs)
songs.head()

## 1-2. 태그 정리 (train)
- 필요한 칼럼(tags, songs)만 남기고 정리
- 리스트 형태로 되어 있는 tag를 문자열로 바꿈

In [ ]:
train.drop(['id', 'plylst_title', 'like_cnt', 'updt_date'], axis = 1, inplace = True)
train['tags'] = [', '.join(train['tags'][i]) for i in range(len(train))]
train.head()

### 1-3. 노래 정리 (song meta)
- 필요한 칼럼 (dtl_gnr, song, gnr, song id)만 남기고 정리
- 리스트 형태로 되어있던 세부 장르와 장르를 문자열로 변경

In [ ]:
songs.drop(['issue_date', 'album_name', 'album_id', 'artist_id_basket', 'artist_name_basket'], axis = 1, inplace = True)
songs['song_gn_dtl_gnr_basket'] = [', '.join(songs['song_gn_dtl_gnr_basket'][i]) for i in range(len(songs))]
songs['song_gn_gnr_basket'] = [', '.join(songs['song_gn_gnr_basket'][i]) for i in range(len(songs))]

songs.head()

## 1-4. 노래 - 태그 매핑 (song_tag_gnr_map.csv 생성)
- 플레이리스트의 tag, song id 매핑
- train 데이터와 song 데이터 합해서 새로운 데이터 생성

In [ ]:
# 플레이리스트 tag와 songs 추출
plylst_song_map = train

plylst_song_map_unnest = np.dstack(
    (
        np.repeat(plylst_song_map.tags.values, list(map(len, plylst_song_map.songs))), 
        np.concatenate(plylst_song_map.songs.values)
    )
)

plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
plylst_song_map['tags'] = plylst_song_map['tags'].astype(str)
plylst_song_map['song_id'] = plylst_song_map['songs'].astype(str)

del plylst_song_map_unnest

In [ ]:
plylst_song_map.drop('songs', axis = 1, inplace = True)
plylst_song_map

In [ ]:
# songs 칼럼명 재설정
songs.columns = ['dtl_gnr', 'song_name', 'gnr', 'song_id']

In [ ]:
# song id 숫자형으로 변경
plylst_song_map['song_id'] = pd.to_numeric(plylst_song_map['song_id'])

In [ ]:
### song_tag_gnr_map.csv (이미 만들어 놓음!!)
# 노래 - 태그 매핑한 데이터 생성 
song_tag = pd.merge(songs, plylst_song_map, on = 'song_id', how = 'left')

# 2. song id - tag 매칭, label 매칭

## 2-1. 첫번째 방법 (태그 개수를 세어서 max 태그의 감정 뽑기)
- 노래마다 tag의 개수를 구한다
- 제일 많은 개수를 차지한 tag를 해당 song의 tag로 설정
- tag와 매칭된 감정으로 label 구하기
- -> 추후 nlp 모델에서 출력한 label과 매칭해 노래 추천

### 2-1-1. 앞서 만든 'song_tag' 데이터 활용

In [ ]:
song_tag

In [ ]:
# tag가 없는 노래 제거
df = df[df['tags'].notnull()]

# tag를 song_id 기준으로 묶음
df[['tags', 'song_id']].groupby('song_id').sum() 

In [ ]:
# index 재설정
df = df.reset_index()
df.drop('index', axis = 1, inplace = True)

# tag를 sum하면 두개 단어가 붙어 나와서 tag 마지막에 쉼표 추가
df['tags'] = [df['tags'][i] + ', ' for i in range(len(df))] 
df[['tags', 'song_id']].groupby('song_id').sum() 

In [ ]:
song_tag = df[['tags', 'song_id']].groupby('song_id').sum().reset_index()
song_tag['tags'] = [song_tag['tags'][i][:-2] for i in range(len(song_tag))]

song_tag.head()

### 2-1-2. song id 별로 tag의 개수 확인
- collections import (딕셔너리 반환)

In [ ]:
# tag_count에 딕셔너리 저장
song_tag['tag_count'] =    [dict(Counter(song_tag['tags'][i].split(', '))) for i in range(len(song_tag))]

song_tag.head()

In [ ]:
# 제일 많이 나온 tag를 max_key로 저장
max_key = []
for i in range(len(song_tag)):
    di = song_tag['tag_count'][i]
    max_key.append([k for k,v in di.items() if max(di.values()) == v])

In [ ]:
song_tag['max_key'] = max_key

song_tag.head()

## 2-2. 두번째 방법 (태그 비율 계산)
- 회의에서 정한 사용할 감정 목록 데이터 사용 `tag_8emotion.xlsx`
- 방금 만들었던 `song_tag` 데이터 사용

In [ ]:
tag_p = pd.read_excel('tag_8emotion.xlsx', sheet_name = 'Sheet1')
tag_p.head()

In [ ]:
# 태그 리스트 생성
t_all = list(tag_p['tag'])

# 감정 리스트 생성
emotions = list(tag_p['emotion'].unique())

### 2-2-1. 태그 - 감정 매칭

- 우리가 사용하기로 한 tag만 남김

In [ ]:
df = song_tag
df.head()

In [ ]:
# 우리가 사용하기로 한 tag만 남겨둔 리스트
tag_in = [[df['tags'][i].split(', ')[j] for j in range(len(df['tags'][i].split(', '))) if df['tags'][i].split(', ')[j] in t_all] for i in range(len(df))] 

# 전체 tag 대신 tag_in을 column으로 넣고 우리가 사용하기로 한 tag만 사용
df['tag_in'] = pd.Series(tag_in)

df.head()

### 2-2-2. tag 없는 노래 삭제

In [ ]:
df['tag_in'] = pd.Series([', '.join(df['tag_in'][i]) for i in range(len(df))])
df = df[df['tag_in'] != '']

df.reset_index(inplace = True) 
df.drop('index', axis = 1, inplace = True) 

df.head()

### 2-2-3. 첫번째 방법과 동일하게 tag 개수를 세고, max key 찾기

In [ ]:
|df['tag_in_count'] = [dict(Counter(df['tag_in'][i].split(', '))) for i in range(len(df))]

max_key = []
for i in range(len(df)):
    di = df['tag_in_count'][i]
    max_key.append([k for k,v in di.items() if max(di.values()) == v])

df['in_max_key'] = pd.Series(max_key)
df.head()

### 2-2-4. max tag - 감정 매칭

In [ ]:
df.drop(['tags', 'tag_count', 'max_key'], axis = 1, inplace = True)

df.head()

In [ ]:
# max tag에 맞는 감정 추출
tag_idx = [[list(tag_p['tag']).index(df['in_max_key'][i][j]) for j in range(len(df['in_max_key'][i]))] for i in range(len(df))] 

# 감정 리스트 생성
emotions = [ [tag_p['emotion'][tag_idx[i][j]] for j in range(len(tag_idx[i]))] for i in range(len(df))] 

# df에 감정 칼럼 추가
df['emotion'] = pd.Series(emotions)

df.head()

### 2-2-5. max 감정 추출

In [ ]:
# 감정 개수 세기
df['emotion_cnt'] = [dict(Counter(df['emotion'][i])) for i in range(len(df))] 

# max 감정 추출
max_emotion = []
for i in range(len(df)):
    di = df['emotion_cnt'][i]
    max_emotion.append([k for k,v in di.items() if max(di.values()) == v])

df['max_emotion'] = pd.Series(max_emotion)
df.head()

## 최종 데이터
- column: song id, tag count, tag max, emotion, count, emotion max
- shape : 395365 x 5

In [ ]:
df

# 3. Rating Matrix
(1104 마지막, 1117 정리 필)
## 3-1. 태그별 가중치 계산
- song id, tag, emotion만 있는 상태에서 시작
- softmax 함수 정의
- song id 별로 감정이 차지하는 비율 계산
- matrix 형태로 생성

In [ ]:
# song id, tag, emotion만 남김
df.drop(['tag max', 'emotion max', 'count'], axis = 1, inplace = True)
df['tag count'] = [dict(Counter(df['tag'][i].split(', '))) for i in range(len(df))] 

In [ ]:
# 태그 비율 계산하는 softmax 함수 생성
def softmax(t):
    s = set(t)
    c = [dict(Counter(t))[name] for name in s]
    return dict(zip(s, np.array(c) / sum(c)))

## 3-2. Matrix 기본 형태

In [ ]:
# 기존 df를 transpose한 형태로 새로운 rating matrix 생성
rating = df.T
# 감정 기준으로 새로 matrix를 만들기 때문에 삭제
rating.drop('tag', inplace = True)
# column을 첫번째 행인 song id로 설정
rating.rename(columns = rating.iloc[0], inplace = True)
# 첫번째 행 삭제
rating.drop('song id', inplace = True)

In [ ]:
rating

## 3-3. 감정 비율 계산

In [ ]:
idx = ['우울한', '울고싶은', '긴장되는', '무서운', '잔잔한', '행복한', '경쾌한', '편안한']
cols = list(rating.columns) 

In [ ]:
all = []

for c in cols:
    s = softmax(rating[c]['emotion'].split(', '))
    emotions = []

    for i in idx:
        try:
            emotions.append(s[i])
        except:
            emotions.append(0)

    # 감정 점수 다 모아서 한번에 column별로 넣을 예정
    all.append(emotions)

## 3-4. 노래 - 감정 Rating Matrix 생성
- index를 감정으로 설정
- column은 song id

In [ ]:
emotion = pd.DataFrame(all).T
emotion.index = idx
emotion.columns = rating.columns

emotion